# Table of distances, proper and coordinate time

In [16]:
# Light years to calculate

distances: list[int] = [
    1,
    2,
    3,
    4,
    5,
    10,
    50,
    100,
    200,
    500,
    # 8, 10, 12, 14, 16, 20, 25,
    # 30, 40, 50, 60, 70, 80, 90,
    # 100, 200, 300, 400, 500,
    1000,
    2000,
    5000,
    10000,
    20000,
    50000,
    100000,
    500000,
    1000000,
    2500000,
    5000000,
    10000000,
    25000000,
]

In [17]:
from mpmath import mp
import relativity_lib as rl
from prettytable import PrettyTable  # , TableStyle

# use 100 decimal places
rl.configure(300)
table = PrettyTable(["Ly", "Prop Yrs", "Co'rd Yrs", "Peak c", "Lorentz", "Apparent c"])
table.align = "r"  # right align everything


def process_dist(distance_ly, flip: bool) -> None:
    accel = "1"
    distance = rl.light_year * rl.ensure(distance_ly)
    a = rl.g * mp.mpf(accel)

    if flip:
        # seconds to burn to get required distance (proper time)
        burn_sec = rl.relativistic_time_for_distance(
            a, distance / 2
        )  # accelerate half-way

        # convert to years
        burn_years = burn_sec * 2 / rl.seconds_per_year

        # coordinate time (outside observer
        coord_years = rl.coordinate_time(a, burn_sec) * 2 / rl.seconds_per_year

        # work out top speed as fraction of light speed
        peak_velocity = rl.relativistic_velocity(a, burn_sec) / rl.c

        # peak lorentz factor
        peak_lorentz = rl.lorentz_factor(peak_velocity * rl.c)
    else:
        # seconds to burn to get required distance (proper time)
        burn_sec = rl.relativistic_time_for_distance(a, distance)

        # convert to years
        burn_years = burn_sec / rl.seconds_per_year

        # coordinate time (outside observer
        coord_years = rl.coordinate_time(a, burn_sec) / rl.seconds_per_year

        # work out top speed as fraction of light speed
        peak_velocity = rl.relativistic_velocity(a, burn_sec) / rl.c

        # peak lorentz factor
        peak_lorentz = rl.lorentz_factor(peak_velocity * rl.c)

    apparent_speed = (
        rl.ensure(distance_ly) / burn_years
    )  # apparent speed in proper years per light year

    table.add_row(
        [
            rl.format_mpf(distance_ly, 0),
            rl.format_mpf(burn_years, 2),
            rl.format_mpf(coord_years, 1),
            rl.format_mpf_significant(peak_velocity, 2, "9"),
            rl.format_mpf(peak_lorentz, 1),
            rl.format_mpf(apparent_speed, 2),
        ]
    )

## Half way flip

In [18]:
for distance in distances:
    process_dist(distance, True)

print(table)

+------------+----------+--------------+--------------------+--------------+------------+
|         Ly | Prop Yrs |    Co'rd Yrs |             Peak c |      Lorentz | Apparent c |
+------------+----------+--------------+--------------------+--------------+------------+
|          1 |     1.89 |          2.2 |               0.75 |          1.5 |       0.52 |
|          2 |     2.58 |          3.4 |               0.87 |          2.0 |       0.77 |
|          3 |     3.07 |          4.5 |              0.919 |          2.5 |       0.97 |
|          4 |     3.45 |          5.6 |              0.945 |          3.0 |       1.15 |
|          5 |     3.77 |          6.6 |              0.960 |          3.5 |       1.32 |
|         10 |     4.85 |         11.7 |              0.986 |          6.1 |       2.06 |
|         50 |     7.71 |         51.9 |            0.99930 |         26.8 |       6.48 |
|        100 |     9.02 |        101.9 |            0.99981 |         52.6 |      11.08 |
|        2

## Non-stop (full burn)

In [19]:
table.clear_rows()
for distance in distances:
    process_dist(distance, False)

print(table)

+------------+----------+--------------+---------------------+--------------+--------------+
|         Ly | Prop Yrs |    Co'rd Yrs |              Peak c |      Lorentz |   Apparent c |
+------------+----------+--------------+---------------------+--------------+--------------+
|          1 |     1.29 |          1.7 |                0.87 |          2.0 |         0.77 |
|          2 |     1.72 |          2.8 |               0.945 |          3.0 |         1.15 |
|          3 |     2.02 |          3.8 |               0.969 |          4.0 |         1.48 |
|          4 |     2.24 |          4.8 |               0.980 |          5.1 |         1.78 |
|          5 |     2.42 |          5.8 |               0.986 |          6.1 |         2.06 |
|         10 |     3.02 |         10.9 |              0.9960 |         11.3 |         3.31 |
|         50 |     4.51 |         50.9 |             0.99981 |         52.6 |        11.08 |
|        100 |     5.17 |        100.9 |            0.999953 |        

In [20]:
rl.configure(200)

initial = mp.mpf("299792457.9999999")
rapidity = rl.rapidity_from_velocity(initial)
doubled = rapidity * 2
velocity = rl.velocity_from_rapidity(doubled)

print(f"Initial velocity: {rl.format_mpf(initial, -1)}")
print(f"Rapidity: {rl.format_mpf(rapidity, 5)}")
print(f"Doubled: {rl.format_mpf(doubled, 5)}")
print(f"Velocity: {rl.format_mpf_significant(velocity, 3, '9')}")

# Rust with astro-float results for comparison:
# Initial velocity = 299,792,457.9999999 (from string)
# Rapidity = 18.16491
# Doubled rapidity = 36.32983
# Doubled vel = 299,792,457.9999999999999999999999833

Initial velocity: 299,792,457.9999999
Rapidity: 18.16492
Doubled: 36.32984
Velocity: 299,792,457.9999999999999999999999833
